## Training Model

We call our trainig_script.py to execute our training offline.Below is for reference.We train the model for 10 epochs currently.WE store the trained weights and training history per epoch.

In [5]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jan  4 21:50:15 2020

@author: dhamuk
"""

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding,BatchNormalization, Dropout, TimeDistributed, Dense,Concatenate, RepeatVector, Activation, Input, add
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input
from tqdm import tqdm
from keras.preprocessing import image
from keras import backend 
from keras.models import load_model
from PIL import Image
import numpy as np
import pandas as pd
import pickle
import Utils as util
from keras.utils import to_categorical



###set path
root_path_dataset = 'Y:/Flickr_Data/'
root_path_features = 'Y:/lab2/ShowTell_Vinit_5thMorn/showtell/data'



###load training description dataset ######
trainfile=root_path_dataset+'Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'
train = util.loadDataset(trainfile)

###load encoded photo features ####
features=pickle.load(open(root_path_features+'/featuresNew.pkl', 'rb'))
print('Features length :',len(features))

####get descriptions
descriptions_map = util.getDescriptions_dataset(root_path_features+'/descriptions.txt',train)
samples=len(descriptions_map)
print('Number of samples :',samples)
all_words=util.desc_list(descriptions_map)
no_words=len(all_words)
print('Number of words :' ,no_words)

### read vocab file ###
vocab= [] 
vocab = list(set(vocab))
vocab= pickle.load(open(root_path_features+'/vocab_new.p', 'rb'))
vocab_size = len(vocab)
print('Vocab size : ',vocab_size)
###

### max caption size
max_length = util.max_length(descriptions_map)
print('max length caption : ',max_length)

### train and dump tokenizer
tokenizer = util.fit_tokenizer(descriptions_map)
pickle.dump( tokenizer, open( "tokernizer.p", "wb" ) )
###
    

      
          
       

    
   



Features length : 8091
Number of samples : 6000
Number of words : 30000
Vocab size :  9632
max length caption :  40


In [6]:
#### this method creates the instances with image,input word and next word as output
def createInstancesByWords(max_length, desc_list, photo):
    img_in1,word_in2, word_y = list(), list(), list()	
    
    for desc in desc_list:
		# get embeddings
        word_seq = tokenizer.texts_to_sequences([desc])[0]       
		
        for i in range(1, len(word_seq)):           
			
            in_seq, out_seq = word_seq[:i], word_seq[i] #break into words sequence till now and the next word			
            in_seq = sequence.pad_sequences([in_seq], maxlen=max_length)[0] # pad			
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0] # embed outout
			#create list 
            img_in1.append(photo)
            word_in2.append(in_seq)
            word_y.append(out_seq)
            
    return np.asarray(img_in1),np.asarray(word_in2),np.asarray(word_y)
                
                
    

### Create batches for training
def data_generator():
    
    while 1:
        ##for all images get img and captions
        for key, caption in descriptions_map.items():              
           
            photo = features[key] #get feature for img
            in1,in2,out=createInstancesByWords( max_length,caption,photo ) #get instances of all words sequences
            
            yield [[in1, in2], out]


In [7]:


## make model
def get_model(input_dim=2048):
    
    #image    
    embed_dim = 300 
    image_inputs = Input(shape=(input_dim,))
    img_layer = Dropout(0.5)(image_inputs)
    img_model_D = Dense(256, activation='relu')(img_layer)
    #sentence
    caption_in = Input(shape=(max_length,))
    caption_embed = Embedding(vocab_size, embed_dim, mask_zero=True)(caption_in)
    caption_dropout = Dropout(0.5)(caption_embed)
    lstm_model = LSTM(256)(caption_dropout)
    #combine inputs to decoder
    decoder = add([img_model_D, lstm_model])
    decoder_final = Dense(256, activation='relu')(decoder)
    #output layer
    output = Dense(vocab_size, activation='softmax')(decoder_final)
    #final model
    model = Model(inputs=[image_inputs, caption_in], outputs=output)    
    model.summary()
    
    return model
    
    

    

In [8]:

#####training #####
    
epoch = 10
#batch_size = 512
model=get_model()
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
modelName='showTellFulltrain_12jan'

### train per epoch

for i in range(epoch): 	
     generator = data_generator() #generate data for epoch
     #train
     history=model.fit_generator(generator, epochs=1, steps_per_epoch=samples, verbose=1)
     #save
     model.save(modelName + str(i) + '.h5')  
     pickle.dump( history.history, open( "hist"+ str(i)+".p", "wb" ) )
    

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 40, 300)      2889600     input_6[0][0]                    
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 2048)         0           input_5[0][0]                    
____________________________________________________________________________________________

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
  10/6000 [..............................] - ETA: 51:45 - loss: 9.1620 - accuracy: 0.0000e+0 - ETA: 37:23 - loss: 9.1373 - accuracy: 0.0320    - ETA: 33:11 - loss: 8.9983 - accuracy: 0.069 - ETA: 30:34 - loss: 8.9263 - accuracy: 0.084 - ETA: 28:39 - loss: 8.8010 - accuracy: 0.088 - ETA: 27:03 - loss: 8.6124 - accuracy: 0.096 - ETA: 26:24 - loss: 8.3869 - accuracy: 0.102 - ETA: 26:02 - loss: 8.2219 - accuracy: 0.098 - ETA: 26:05 - loss: 8.1170 - accuracy: 0.103 - ETA: 25:23 - loss: 7.9550 - accuracy: 0.1091

KeyboardInterrupt: 